In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.model_selection import train_test_split
from lazypredict.Supervised import LazyRegressor
from sklearn.ensemble import ExtraTreesRegressor, RandomForestRegressor, GradientBoostingRegressor, BaggingRegressor
from xgboost import XGBRegressor


In [2]:
df_nasabah = pd.read_csv(r"C:\Users\DHONI HANIF\OneDrive\Documents\AI Collection and Loss Reverse Forecast\data_preparation\EDA\Univariate_bivariate_multivariate\Univariate_Analysis\new_nasabah.csv")
df_nasabah.head()

,Unnamed: 0,debtor_gender,age,debtor_education_level,debtor_martial_status,employment_type,number_of_dependents,net_income,payment_date,loan_amount,avg_amounts_previous_bills,avg_previous_payment,amount_of_late,late_payment_amount,credit_risk,credit_score,arrears_amounts,aset,loss_reverse,aging
0,0,Wanita,80,D3,Sudah menikah,Retired,3,15500000.00,4,15625000.00,29500000,24350000,2,1,Tertagih,670,0,11000000,0.00,DPK
1,1,Pria,44,D3,Sudah menikah,Full-time,3,17500000.00,8,17708333.33,3000000,1600000,31,3,Tertagih,402,1,17000000,17708333.33,DPK
2,2,Pria,35,D2,Sudah menikah,Full-time,4,17500000.00,8,14833333.33,13500000,9050000,0,0,Tertagih,505,0,14500000,0.00,Lancar
3,3,Wanita,53,D2,Sudah menikah,Retired,3,15500000.00,7,15291666.67,10500000,8350000,64,5,Tertagih,508,1,11500000,15291666.67,DPK
4,4,Pria,24,SMA,Sudah menikah,Contract,5,8700000.00,9,14041666.67,6500000,5750000,14,2,Tertagih,371,0,5500000,0.00,DPK


In [3]:
columns = {}
for i in df_nasabah.columns:
    if df_nasabah[i].dtype == "category" or df_nasabah[i].dtype == "object":
        columns[i] = LabelEncoder().fit(df_nasabah[i])
        df_nasabah[i] = columns[i].transform(df_nasabah[i])
        
scaler = RobustScaler().fit(df_nasabah[[i for i in df_nasabah.columns if i not in columns]])
X = pd.DataFrame(scaler.transform(df_nasabah[[i for i in df_nasabah.columns if i not in columns]]), columns=df_nasabah[[i for i in df_nasabah.columns if i not in columns]].columns)
for i in columns:
    X[i] = df_nasabah[i]
X.head(5)

,Unnamed: 0,age,number_of_dependents,net_income,payment_date,loan_amount,avg_amounts_previous_bills,avg_previous_payment,amount_of_late,late_payment_amount,credit_score,arrears_amounts,aset,loss_reverse,debtor_gender,debtor_education_level,debtor_martial_status,employment_type,credit_risk,aging
0,-1.00,1.00,0.00,0.00,-0.25,0.19,1.24,1.43,-0.80,-0.60,1.59,-1.00,0.60,-0.93,1,2,1,5,0,0
1,-1.00,-0.16,0.00,0.27,0.75,0.98,-0.88,-0.88,-0.41,-0.20,0.37,0.00,1.80,0.12,0,2,1,1,0,0
2,-1.00,-0.45,0.50,0.27,0.75,-0.11,-0.04,-0.12,-0.83,-0.80,0.84,-1.00,1.30,-0.93,0,1,1,1,0,3
3,-1.00,0.13,0.00,0.00,0.50,0.06,-0.28,-0.19,0.03,0.20,0.85,0.00,0.70,-0.02,1,1,1,5,0,0
4,-1.00,-0.81,1.00,-0.91,1.00,-0.41,-0.60,-0.46,-0.64,-0.40,0.22,-1.00,-0.50,-0.93,0,7,1,0,0,0


In [4]:
df2 = X.drop(["Unnamed: 0","credit_risk"], axis=1)


In [5]:
X2 = df2.drop("loss_reverse", axis=1)
y2 = df2["loss_reverse"]

X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=.2, random_state=42)
reg = LazyRegressor(verbose=0, ignore_warnings=True)
models, predictions = reg.fit(X2_train, X2_test, y2_train, y2_test)
models

  0%|          | 0/42 [00:00<?, ?it/s]

100%|██████████| 42/42 [00:21<00:00,  1.96it/s]

[LightGBM] [Warning] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000685 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1295
[LightGBM] [Info] Number of data points in the train set: 4000, number of used features: 17
[LightGBM] [Info] Start training from score 0.343097


,Adjusted R-Squared,R-Squared,RMSE,Time Taken
Model,,,,
ExtraTreesRegressor,1.00,1.00,0.03,2.02
RandomForestRegressor,1.00,1.00,0.03,4.29
BaggingRegressor,1.00,1.00,0.04,0.45
DecisionTreeRegressor,1.00,1.00,0.05,0.07
GradientBoostingRegressor,1.00,1.00,0.05,1.01
ExtraTreeRegressor,1.00,1.00,0.05,0.04
LGBMRegressor,1.00,1.00,0.06,0.17
HistGradientBoostingRegressor,1.00,1.00,0.06,0.62
MLPRegressor,1.00,1.00,0.07,1.17


In [6]:
def evaluate_performa(X_test, y_test, model1, model2, model3, model4, model5):
    rate = [0, 0, 0, 0, 0]
    #difference = [0, 0, 0, 0, 0]

    X_test = X_test.reset_index(drop=True)
    for i, _ in X_test.iterrows():
        X = np.array(X_test.iloc[[i]])
        y = np.array(y_test.iloc[[i]])
        difference = [0, 0, 0, 0, 0]
        difference[0] = abs(model1.predict(X) - y[0])
        difference[1] = abs(model2.predict(X) - y[0])
        difference[2] = abs(model3.predict(X) - y[0])
        difference[3] = abs(model4.predict(X) - y[0])
        difference[4] = abs(model5.predict(X) - y[0])
        print(difference)

        a = int(np.argmin(difference))
        print(a)
        for i in range(len(difference)):
            if difference[a] == difference[i]:
                rate[i] += 1
    
    a = f"Best model : {rate.index(max(rate))+1}"
    return a, rate

In [7]:
extra = ExtraTreesRegressor()
forest = RandomForestRegressor()
xgb = XGBRegressor()
gradient = GradientBoostingRegressor()
bagging = BaggingRegressor()


extra.fit(X2_train, y2_train)
forest.fit(X2_train, y2_train)
xgb.fit(X2_train, y2_train)
gradient.fit(X2_train, y2_train)
bagging.fit(X2_train, y2_train)

a = evaluate_performa(X2_test, y2_test, extra, forest, xgb, gradient, bagging)
a

[array([3.66373598e-15]), array([2.22044605e-15]), array([0.00067335], dtype=float32), array([0.00688304]), array([1.44328993e-15])]
4
[array([0.00014851]), array([1.80411242e-16]), array([0.00124556], dtype=float32), array([0.00062003]), array([2.77555756e-17])]
4
[array([3.55271368e-15]), array([1.99840144e-15]), array([0.00172436], dtype=float32), array([0.01382217]), array([1.44328993e-15])]
4
[array([0.00445545]), array([0.00274752]), array([0.00688362], dtype=float32), array([0.02549302]), array([0.00742574])]
1
[array([0.00017327]), array([0.00019802]), array([0.00217819], dtype=float32), array([0.00900558]), array([0.00148515])]
0
[array([0.00054455]), array([0.00066832]), array([0.0015689], dtype=float32), array([0.0034234]), array([0.00024752])]
4
[array([0.00034653]), array([0.00054455]), array([0.00306702], dtype=float32), array([0.02055709]), array([2.22044605e-16])]
4
[array([0.00423267]), array([0.00054455]), array([0.00231977], dtype=float32), array([0.00072823]), array

('Best model : 5', [162, 218, 36, 23, 624])